In [29]:
import helpers
import numpy as np
import re
import math
from tqdm import tqdm
import copy
import networkx as nx
from collections import Counter
import ast
import itertools as it

data = """on x=-20..26,y=-36..17,z=-47..7
on x=-20..33,y=-21..23,z=-26..28
on x=-22..28,y=-29..23,z=-38..16
on x=-46..7,y=-6..46,z=-50..-1
on x=-49..1,y=-3..46,z=-24..28
on x=2..47,y=-22..22,z=-23..27
on x=-27..23,y=-28..26,z=-21..29
on x=-39..5,y=-6..47,z=-3..44
on x=-30..21,y=-8..43,z=-13..34
on x=-22..26,y=-27..20,z=-29..19
off x=-48..-32,y=26..41,z=-47..-37
on x=-12..35,y=6..50,z=-50..-2
off x=-48..-32,y=-32..-16,z=-15..-5
on x=-18..26,y=-33..15,z=-7..46
off x=-40..-22,y=-38..-28,z=23..41
on x=-16..35,y=-41..10,z=-47..6
off x=-32..-23,y=11..30,z=-14..3
on x=-49..-5,y=-3..45,z=-29..18
off x=18..30,y=-20..-8,z=-3..13
on x=-41..9,y=-7..43,z=-33..15
on x=-54112..-39298,y=-85059..-49293,z=-27449..7877
on x=967..23432,y=45373..81175,z=27513..53682"""

with open("input_22.txt", 'r') as f:
    data = f.read()

data = data.lstrip()
data = data.rstrip()
    
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
data = re.findall("(\w+) x=([-+]?\d+)..([-+]?\d+),y=([-+]?\d+)..([-+]?\d+),z=([-+]?\d+)..([-+]?\d+)", data)
#data = [x.rstrip() for x in data.split("\n\n") if x]
data


[('on', '-30', '22', '-28', '20', '-17', '37'),
 ('on', '-20', '34', '-30', '17', '-30', '21'),
 ('on', '-7', '42', '-23', '26', '-18', '29'),
 ('on', '-8', '44', '-34', '20', '-14', '32'),
 ('on', '-13', '32', '-47', '6', '-38', '6'),
 ('on', '-48', '6', '-36', '14', '-18', '33'),
 ('on', '-31', '13', '-22', '32', '-44', '9'),
 ('on', '-18', '31', '-48', '-2', '-18', '28'),
 ('on', '-9', '38', '-15', '35', '-48', '6'),
 ('on', '-14', '35', '-42', '6', '-9', '43'),
 ('off', '25', '40', '-21', '-12', '3', '18'),
 ('on', '-32', '12', '-28', '18', '-12', '40'),
 ('off', '-12', '1', '27', '38', '5', '18'),
 ('on', '-34', '14', '-38', '13', '-29', '23'),
 ('off', '9', '28', '-35', '-26', '7', '16'),
 ('on', '-18', '27', '-21', '27', '-42', '7'),
 ('off', '18', '32', '26', '42', '-8', '10'),
 ('on', '-15', '34', '-30', '17', '-19', '33'),
 ('off', '12', '23', '32', '45', '26', '41'),
 ('on', '-36', '13', '-33', '13', '-27', '17'),
 ('on', '-8328', '22599', '-70363', '-44520', '-57360', '-369

In [30]:
coords = set()

for x in tqdm(data):
    opp, x1, x2, y1, y2, z1, z2 = x
    x1,x2,y1,y2,z1,z2 = int(x1),int(x2),int(y1),int(y2),int(z1),int(z2)
    if opp == "on":
        for x in range(x1,x2+1):
            for y in range(y1,y2+1):
                for z in range(z1,z2+1):
                    if -50 <= x <= 50 and -50 <= y <= 50 and -50 <= z <= 50:
                        coords.add((x,y,z))
    elif opp == "off":
        for x in range(x1,x2+1):
            for y in range(y1,y2+1):
                for z in range(z1,z2+1):
                    if -50 <= x <= 50 and -50 <= y <= 50 and -50 <= z <= 50:
                        coords.discard((x,y,z))
print(coords)

  5%|███▊                                                                             | 20/420 [00:09<03:09,  2.11it/s]


KeyboardInterrupt: 

In [31]:
len(coords)
#range(-10,10)

503864

In [ ]:
class grid3D:
    def __init__(self, size):
        self.size = size
        self.grid = np.zeros(size).astype(int)
    def clear(self):
        self.grid = np.zeros(self.size).astype(int)
    def on(self,p1,p2, binary=True):
        x1,y1 = int(p1[0]), int(p1[1])
        x2,y2 = int(p2[0]), int(p2[1])
        if binary:
            self.grid[x1:x2+1, y1:y2+1] = 1
        else:
            self.grid[x1:x2+1, y1:y2+1] += 1
    def off(self,p1,p2, binary=True):
        x1,y1 = int(p1[0]), int(p1[1])
        x2,y2 = int(p2[0]), int(p2[1])
        
        if binary:
            self.grid[x1:x2+1, y1:y2+1] = 0
        else:
            self.grid[x1:x2+1, y1:y2+1] -= 1   
            self.grid[x1:x2+1, y1:y2+1] = self.grid[x1:x2+1, y1:y2+1].clip(min=0)
    def toggle(self,p1,p2, binary=True):
        x1,y1 = int(p1[0]), int(p1[1])
        x2,y2 = int(p2[0]), int(p2[1])
        sli = self.grid[x1:x2+1, y1:y2+1]
        if binary:
            self.grid[x1:x2+1, y1:y2+1] = np.where((sli==0)|(sli==1), sli^1, sli)
        else:
            self.grid[x1:x2+1, y1:y2+1] += 2
    def toggle_all(self):
        self.grid = np.where((self.grid==0)|(self.grid==1), self.grid^1, self.grid)
    def nPrint(self):
        npNicePrint(self.grid)
    def countOn(self):
        return sum(sum(self.grid))
    def countOff(self):
        return (self.size[0] * self.size[1]) - sum(sum(self.grid))
    def save(self, name):
        im = Image.fromarray(np.uint8(self.grid * 255) , 'L')
        #im = Image.fromarray(self.grid).convert('1')
        im.save(f"{name}.png")